# Homework 5: Data modeling and SQL for IMDB data

***
## SQL Database Design

As mentioned in the google doc, star schemas are used commonly for data warehouses. Wikipedia has a short and clear [Star Schema](https://en.wikipedia.org/wiki/Star_schema) article.  This [reference](https://www.guru99.com/star-snowflake-data-warehousing.html) is also good, and describes both star schemas and their connection to snowflake schemas.

In this part of the assignment, you will develop a **star schema**.

Tips:

1. **Working with flat files/raw data**
   When working with flat files of data it's important to check them out in their raw form (i.e. using a text editor or unix head command if the files are too big for your editor). For tabular data you should review every column and make sure you understand what it represents.  

   For this assignment, we highly recommend you create a markdown cell with the name of every column that is currently in the file. (Copy and paste can be your friend here.)  Often times it you may need to fix the csv file before you import it. 

2. As you do this, sometimes there obvious poblems become apparent, such as header rows that need to skipped over (or deleted), and column names that are inappropriate for the system you are using. 

4. The file 'all_data_raw.csv' has two problems: extra lines at the beginning and inconsitent column names) that you should fix using the text editor. Column names should not contain spaces and should also be consistent.

3. Create a new corrected version of the data file named 'all_data.csv'.

Below is some code that will read an **appropiately** formated csv file and display its contents.  The strings in `all_data.csv` use the `latin1` character encoding. 

**new names**    
director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,  
actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,  
actor_3_name,facenumber_in_poster,num_user_for_reviews,language,country,content_rating,budget,  
title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('all_data.csv',delimiter= ',',encoding='latin1')
# print("shape:",df.shape)
df.head()

,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Gore Verbinski,302.0,169.0,563,1000,Orlando Bloom,40000,309404152.0,Action,Johnny Depp,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000,7.1,2.35,0
1,Christopher Nolan,813.0,164.0,22000,23000,Christian Bale,27000,448130642.0,Action,Tom Hardy,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000,8.5,2.35,164000
2,Andrew Stanton,462.0,132.0,475,530,Samantha Morton,640,73058679.0,Action,Daryl Sabara,...,738.0,English,USA,PG-13,263700000.0,2012.0,632,6.6,2.35,24000
3,Nathan Greno,324.0,100.0,15,284,Donna Murphy,799,200807262.0,Adventure,Brad Garrett,...,387.0,English,USA,PG,260000000.0,2010.0,553,7.8,1.85,29000
4,David Yates,375.0,153.0,282,10000,Daniel Radcliffe,25000,301956980.0,Adventure,Alan Rickman,...,973.0,English,UK,PG,250000000.0,2009.0,11000,7.5,2.35,10000


In [3]:
print(df.dtypes)

director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes        int64
actor_3_facebook_likes         int64
actor_2_name                  object
actor_1_facebook_likes         int64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes         int64
imdb_score                   float64
aspect_ratio                 float64
movie_facebook_likes           int64
dtype: object


Note: 24 columns should be read in from the the csv file.

### Task 5.1
Draw a star schema for this dataset, save your drawing as an image and display it in the the cell below.

1. Your star schema should be composed of one fact table and three dimension tables. 
1. For each table, underline the primary key.
1. Specify data types for each item.
1. Draw 1:N, N:1, N:N or 1:1 on connecting line.
1. Including image in markdown is easy, see [here](https://stackoverflow.com/questions/255170/markdown-and-image-alignment).

Your final drawing should be similar to this [Schema Drawing Example](https://docs.google.com/drawings/d/1Bd1l9LGJZnNQD6BTcZtLHsxWW1a3v07zuUYzz0TyZOk/edit?usp=sharing). 

For this assignment, your fact table should only contain references to other facts. Sometimes 1-1 relationships are collapsed into the fact table, as is seen [here](https://www.guru99.com/images/1/022218_0758_StarandSnow1.png)


Note: We recommended using ipe of Google Drawings. Google Drawngs is an online image creator that can be used to draw the Schema, and is what the TAs used to create the Schema in the solution.

***Solution***
Insert your schema drawing here. (Save your drawing as movie_star_jpg)

![Movie Star Schema](movie_star_schema.jpg)

### Design database schmea
 
In this part, we will create a [SQLite3 database](https://docs.python.org/3/library/sqlite3.html).

The first thing you'll need to do when interacting with a SQLite database (and generally any SQL databases) is to open up a connection (see below). If the database doesn't exist, the SQLite3 module will automatically create one (which can be helpful but also dangerous if your code is designed to automatically create a new instance of a database if one doesn't exist at the current connection).

Resources:
Use SQLite REPL, the [API documentation](https://docs.python.org/3/library/sqlite3.html) and the [SQLite Tutorial](https://www.sqlitetutorial.net/).


In [4]:
import sqlite3
import csv

# creates a connection to the db. 
# This creates/re-opens the corresponding db file on disk because SQLite3 is a file-based databse.
conn = sqlite3.connect('movie.db')    

### Task 5.2 'create_database'
Fill in function `create_database` with the following instructions.

- [Drop all existing tables](https://www.techonthenet.com/sqlite/tables/drop_table.php) in the database. Doing this will allow us to simply re-run the script if we want to try a new schema.
- [Create the tables](http://www.sqlitetutorial.net/sqlite-create-table) according to your star schema. 

**Note:** 

- To execute queries, simply run the `cursor.execute()` with your query as the argument.
- Remember to to run `conn.commit()` in order to commit (i.e. save) your changes.
- Remember to define `PRIMARY KEY`s and `FOREIGN KEY`s in your table, refer to [here](http://www.sqlitetutorial.net/sqlite-primary-key/).
- Remember to commit the connection to save changes.
- In SQLite, `INTEGER PRIMARY KEY` column is auto-incremented and is thus good for IDs
- You may assume the names of actors and directors uniquely identify them

In [5]:
def create_database(conn):
    """
    Using the database connection conn, create tables appropriate for 
    storing and using IMDB data.  Drops any pre-exsiting tables that would be re-defined by this procedure.
    
    Arguments:
        conn: database connection
        c: a cursor on `
    """
    # TODO: implement me'
    c = conn.cursor()
    
    c.executescript('''
    DROP TABLE IF EXISTS actors;
    DROP TABLE IF EXISTS directors;
    DROP TABLE IF EXISTS movies;
    DROP TABLE IF EXISTS facts;
    
    PRAGMA foreign_keys=5;
    
    CREATE TABLE actors(
        actor_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        actor_name TEXT UNIQUE,
        actor_fb_likes INTEGER);
        
    CREATE TABLE directors(
        director_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        director_name TEXT UNIQUE,
        director_fb_likes INTEGER);
        
    CREATE TABLE movies(
        movie_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        num_critic_for_reviews REAL,
        duration REAL, 
        gross REAL,
        genres REAL,
        movie_title TEXT,
        num_voted_users INTEGER,
        cast_total_fb_likes INTEGER,
        facenumber_in_poster REAL,
        num_user_for_reviews INTEGER,
        language TEXT,
        country TEXT,
        content_rating REAL,
        budget REAL,
        title_year INTEGER,
        imdb_score REAL,
        aspect_ratio REAL,
        movie_fb_likes INTEGER);
        
    CREATE TABLE facts(
        movie_id INTEGER NOT NULL,
        director_id INTEGER NOT NULL,
        actor_1_id INTEGER NOT NULL,
        actor_2_id INTEGER NOT NULL,
        actor_3_id INTEGER NOT NULL,
        FOREIGN KEY(director_id) REFERENCES directors(director_id),
        FOREIGN KEY(movie_id) REFERENCES movies(movie_id),
        FOREIGN KEY(actor_1_id) REFERENCES actors(actor_id),
        FOREIGN KEY(actor_2_id) REFERENCES actors(actor_id),
        FOREIGN KEY(actor_3_id) REFERENCES actors(actor_id),
        PRIMARY KEY(movie_id, director_id, actor_1_id, actor_2_id, actor_3_id));
    
    ''')
    
    conn.commit()
    
create_database(conn)

Your new sqlite3 database should now be in your assignment directory. Most databases have simple command line interfaces that let you inspect and manipulate there contents. Use sqlite3's command interface to play around with your database! 

You can connect to a sqlite3 database via terminal with the following command:
```
sqlite3 move.db
```
You can execute arbitrary SQL commands from the `sqlite>` prompt.

To see the schema of your table, you can run the following command in sqlite REPL:
```
.schema
```

More info on your other options can be found here.
```
.help
```

Finally, to exit from a sqlite3 database use the following command:
~~~~
.exit
~~~~

#### Update database

Many database examples begin with different csv files, one for each table. 
However, this isn't how real world data is typically structured when you first receive it. 

Often, you will receive data "dump" files.  As a data scientist, you need to figure out how to split it into different tables, and insert the information into a live database.

~~~~
director_name
num_critic_for_reviews
duration
director_facebook_likes
actor_3_facebook_likes
actor_2_name
actor_1_facebook_likes
gross
genres
actor_1_name
movie_title
num_voted_users
cast_total_facebook_likes
actor_3_name
facenumber_in_poster
num_user_for_reviews
language
country
content_rating
budget
title_year
actor_2_facebook_likes
imdb_score
aspect_ratio
movie_facebook_likes
~~~~

***
### Task 5.3: Update Database
Fill in function `update_database` to insert the csv file contents into the movie database.  You might find the list of column definitions above helpful.  Do not assume the database is empty.  Inserting the exact same data a second time should not change the database.

Tips:   
1. Consider using `INSERT OR IGNORE INTO` where suitable
2. Use helper function to avoid redundant code
3. In order to populate the fact table you will need to query them in order to retrieve the each dimension row id. Here is some SQL code that does this that you are welcome to use.
    ```sql
    INSERT OR IGNORE INTO facts 
    SELECT m.movie_id, d.director_id, a1.actor_id, a2.actor_id, a3.actor_id
    FROM movies m, directors d, actors a1, actors a2, actors a3
    WHERE m.movie_title = ? AND d.director_name = ? AND a1.actor_name = ? 
    AND a2.actor_name = ? AND a3.actor_name = ?
    ```

In [6]:
import csv
import sqlite3


def update_database(conn, csv_file):
    reader = csv.DictReader(open(csv_file, encoding='latin1'))
    c = conn.cursor()
    print(reader.fieldnames)
    # TODO: implement me
    for row in reader:
        # each row is a dictionary, where the column names are the keys
        row = {k: v.lower() for k, v in row.items()} 
#         print(row)
        actors_vals_to_insert = (row['actor_1_name'],row['actor_1_facebook_likes'])
        c.execute('''INSERT OR IGNORE INTO actors (actor_name, actor_fb_likes)
        VALUES (?, ?)''', actors_vals_to_insert)
        
        actors_vals_to_insert = (row['actor_2_name'],row['actor_2_facebook_likes'])
        c.execute('''INSERT OR IGNORE INTO actors (actor_name, actor_fb_likes)
        VALUES (?, ?)''', actors_vals_to_insert)
        
        actors_vals_to_insert = (row['actor_3_name'],row['actor_3_facebook_likes'])
        c.execute('''INSERT OR IGNORE INTO actors (actor_name, actor_fb_likes)
        VALUES (?, ?)''', actors_vals_to_insert)
        
        directors_vals_to_insert = (row['director_name'],row['director_facebook_likes'])
        c.execute('''INSERT OR IGNORE INTO directors (director_name, director_fb_likes)
        VALUES (?, ?)''', directors_vals_to_insert)
        
        movies_vals_to_insert = (row['num_critic_for_reviews'], row['duration'],
                                row['gross'],row['genres'],row['movie_title'],
                                row['num_voted_users'],row['cast_total_facebook_likes'],
                                row['facenumber_in_poster'],row['num_user_for_reviews'],
                                row['language'],row['country'],row['content_rating'],
                                row['budget'],row['title_year'],row['imdb_score'],
                                row['aspect_ratio'],row['movie_facebook_likes'])
        c.execute('''INSERT OR IGNORE INTO movies (num_critic_for_reviews, duration,
        gross, genres, movie_title, num_voted_users, cast_total_fb_likes, facenumber_in_poster,
        num_user_for_reviews, language, country, content_rating, budget, title_year, imdb_score,
        aspect_ratio, movie_fb_likes) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                  movies_vals_to_insert)
        
        fact_vals_to_insert = (row['movie_title'], row['director_name'], row['actor_1_name'], row['actor_2_name'], row['actor_3_name'])
        c.execute('''INSERT OR IGNORE INTO facts 
        SELECT m.movie_id, d.director_id, a1.actor_id, a2.actor_id, a3.actor_id
        FROM movies m, directors d, actors a1, actors a2, actors a3
        WHERE m.movie_title = ? AND d.director_name = ? AND a1.actor_name = ? 
        AND a2.actor_name = ? AND a3.actor_name = ?''', fact_vals_to_insert)
        
        
        conn.commit()
        
                                
# create_database(conn)
update_database(conn, 'all_data.csv')
print("DONE")

['director_name', 'num_critic_for_reviews', 'duration', 'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name', 'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name', 'movie_title', 'num_voted_users', 'cast_total_facebook_likes', 'actor_3_name', 'facenumber_in_poster', 'num_user_for_reviews', 'language', 'country', 'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score', 'aspect_ratio', 'movie_facebook_likes']
DONE


***
### Task 5.4: Running Queries

Now that you have created a database with the IMDB data. Your customer needs records that satisfy their requirements based on several constraints in order to improve their business revenue. 
In this part, you will create several SQL statements to extract them. 

IPython has a built-in module called [sql](https://pypi.python.org/pypi/ipython-sql) which allows us to easily interface with a database. The following two lines load this SQL extension and connect to your IMDB database:

In [7]:
!pip install -q ipython-sql --user

In [8]:
%reload_ext sql
%sql sqlite:///movie.db

'Connected: @movie.db'

Your task is to write 10 queries that fetch certain records for your customers. To run SQL queries, first call <font color = "red">%%sql</font> magic and then write your SQL queries.

**Query 5.4.1: Find the number of directors in the directors table.**


In [9]:
%%sql
SELECT COUNT(DISTINCT(director_name)) FROM directors

 * sqlite:///movie.db
Done.


COUNT(DISTINCT(director_name))
1453



**Query 5.4.2: Find the number of actors in the actors table.**


In [10]:
%%sql
SELECT COUNT(DISTINCT(actor_name)) FROM actors

 * sqlite:///movie.db
Done.


COUNT(DISTINCT(actor_name))
3872



**Query 5.4.3: Find the number of movies in the movies table.**


In [11]:
%%sql
SELECT COUNT(DISTINCT(movie_title)) FROM movies

 * sqlite:///movie.db
Done.


COUNT(DISTINCT(movie_title))
3023



There should be 1453 directors, 3872 actors, 3023 movies. Please make sure you get the same results before running the following queries.

Now, we can have some fun querying the database!

**Query 5.4.4: Find years that have more than 100 movies and arrange them in ascending order.**


In [12]:
%%sql
SELECT title_year, COUNT(title_year)
FROM movies
GROUP BY title_year
Having COUNT(title_year) > 100
ORDER BY title_year ASC

 * sqlite:///movie.db
Done.


title_year,COUNT(title_year)
1999,125
2000,123
2001,147
2002,162
2003,116
2004,146
2005,148
2006,155
2007,120
2008,151



**Query 5.4.5: Order director name from most Facebook likes to fewest Facebook likes and only show top 10 directors.**


In [13]:
%%sql
SELECT director_name, SUM(director_fb_likes)
FROM directors
GROUP BY director_name
ORDER BY SUM(director_fb_likes) DESC
LIMIT 10

 * sqlite:///movie.db
Done.


director_name,SUM(director_fb_likes)
joseph gordon-levitt,23000
christopher nolan,22000
david fincher,21000
denzel washington,18000
kevin spacey,18000
martin scorsese,17000
clint eastwood,16000
quentin tarantino,16000
tom hanks,15000
j.j. abrams,14000



**Query 5.4.6: Find the titles of movies that are directed by Christopher Nolan and order them by movie title.**


In [14]:
%%sql
SELECT m.movie_title AS movie_title
FROM movies m, directors d, facts f
ON f.movie_id = m.movie_id AND f.director_id = d.director_id
WHERE director_name == "christopher nolan"
GROUP BY movie_title

 * sqlite:///movie.db
Done.


movie_title
batman begins
inception
insomnia
interstellar
memento
the dark knight
the dark knight rises
the prestige


**Query 5.4.7: Find all the directors who directed at least 5 movies since 2007 (included) sorted by the number of movies in descending order.**


In [15]:
%%sql
SELECT d.director_name AS director_name, m.title_year AS title_year,COUNT(DISTINCT(m.movie_title)) AS sum_movie_title
FROM movies m, directors d, facts f
ON f.movie_id = m.movie_id AND f.director_id = d.director_id
WHERE m.title_year >= 2007
GROUP BY d.director_name
HAVING sum_movie_title >= 5
ORDER BY sum_movie_title DESC

 * sqlite:///movie.db
Done.


director_name,title_year,sum_movie_title
clint eastwood,2009,7
ethan coen,2008,6
shawn levy,2009,6
steven spielberg,2008,6
woody allen,2011,6
andy fickman,2009,5
antoine fuqua,2013,5
david ayer,2016,5
david fincher,2008,5
dennis dugan,2008,5



**Query 5.4.8: Find the top five most productive actors since 2010 (included).**


In [16]:
%%sql
SELECT a.actor_id, a.actor_name, COUNT(*) AS count FROM actors AS a
JOIN facts AS f ON f.actor_1_id = a.actor_id OR f.actor_2_id = a.actor_id OR f.actor_3_id = a.actor_id
JOIN movies AS m ON m.movie_id = f.movie_id AND m.title_year >= 2010
GROUP BY a.actor_id
ORDER BY count DESC
LIMIT 5

 * sqlite:///movie.db
Done.


actor_id,actor_name,count
43,liam neeson,13
138,steve carell,12
190,bradley cooper,11
205,jennifer lawrence,11
298,channing tatum,11


***
### Query Optimization
In this part, we will see how to optimize database access by indexing and using materialized views. 

The next three functions create and exercise a toy database.
`setup_test` creates a connection to the database.  `insert_test` performs `num` SQL `SELECT`s, the  function `select_test` does `num` SQL `INSERT`s.  

In [17]:
def setup_test(db):
    conn = sqlite3.connect(db)
    conn.execute('DROP TABLE IF EXISTS scripts')
    conn.execute('''
            CREATE TABLE scripts (
                script_id INTEGER,
                author_name text,
                script_name text,
                author_facebook_likes int,
                PRIMARY KEY(script_id))
                ''')
    conn.commit()
    return conn
                

def insert_test(conn, num):
    c = conn.cursor()
    for i in range(num):
        c.execute('''INSERT INTO scripts (script_name) VALUES ('Dr. Who %d')'''%i)
    conn.commit()

def select_test(conn, num):
    c = conn.cursor()
    for i in range(num):
        c.execute('''SELECT * 
                     FROM scripts
                     WHERE script_name = "Dr. Who %d"'''%i)



Below is some code for benchmarking your database.  Run it to see how well your database currently performs.

In [18]:
conn = setup_test('test.db')
insert_test(conn, 15000)

In [19]:
%timeit -n 3 -r 3 insert_test(conn, 15000)

94.4 ms ± 10 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [20]:
%timeit -n 3 -r 3 select_test(conn, 5000) 

830 ms ± 6.31 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


Optimize the database so that `test_select` test performs more quickly.  Fill in the appropriate SQL commands in the cell below.  

Hint: try adding an INDEX. See [here](http://www.sqlitetutorial.net/sqlite-index/) for some information on  `INDEX` and `CREATE INDEX`.

In [21]:
conn1 = setup_test('test1.db')
# YOUR CODE HERE:
c = conn1.cursor()
c.execute('''
    CREATE INDEX idx_test_name
    ON scripts(script_name)''')
conn1.commit()

Run the following code to see how your optimization performs.

In [22]:
%timeit -n 3 -r 3 insert_test(conn1, 15000)

137 ms ± 20.2 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [23]:
%timeit -n 3 -r 3 select_test(conn1, 5000) 

155 ms ± 7.73 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Task 5.5: Index trade-offs
Explain the performance changes you see.  Have any trade-offs been made?

***Solution***    

We can see that the time for insert increases and the time for select decreases. Selecting using index is the same as looking up in the dictionary, thus the time reduces. However, inserting requires updating the dictionary, thus time increases. 

## Running Queries in BigQuery

Go back to the Google Doc for instructions on the BigQuery section of the assignment. 

Add your solutions below.

Your solutions below:

#### Query 5.6.1 Question Count
```python
SELECT COUNT(*) 
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE owner_user_id = 426;
```
![5.6.1](5.6.1.png)

#### Query 5.6.2 Question-answer Pairs
```python
SELECT COUNT(*) 
FROM `bigquery-public-data.stackoverflow.posts_questions` AS q, `bigquery-public-data.stackoverflow.posts_answers` AS a
WHERE a.parent_id = q.id AND q.owner_user_id = 426 AND a.owner_user_id = 2142539;
```
![5.6.2](5.6.2.png)

#### Query 5.6.3 Question to answer ratio
``` python

SELECT total_questions,
      total_pairs,
      total_pairs * 1.0 / total_questions AS ratio
FROM
   -- subquery 1 for # of questions from U
   (SELECT COUNT(*) AS total_questions      
    FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
    WHERE q.owner_user_id = 426),
   -- subquery 2 for # of questions from U answered by V
   (SELECT COUNT(*) AS total_pairs          
    FROM `bigquery-public-data.stackoverflow.posts_answers` AS a,
         `bigquery-public-data.stackoverflow.posts_questions` AS q
    WHERE a.parent_id = q.id
      AND q.owner_user_id = 426
      AND a.owner_user_id = 2142539)
;
```
![5.6.3](5.6.3.png)

#### Query 5.6.4 Top Audience
```python
-- subquery for # of questions from U
WITH questions_count AS (
  SELECT COUNT(*) AS val
  FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
  WHERE q.owner_user_id = 426)
SELECT answer.owner_user_id as answered_by,
       COUNT(*) AS pair_count,
       COUNT(*) / questions_count.val as ratio
FROM questions_count,
     `bigquery-public-data.stackoverflow.posts_answers` AS answer
       JOIN `bigquery-public-data.stackoverflow.posts_questions` AS question
       ON question.id = answer.parent_id AND question.owner_user_id = 426
GROUP BY answer.owner_user_id, questions_count.val
ORDER BY ratio DESC
LIMIT 5;
```
![5.6.4](5.6.4.png)


#### Query 5.6.5 Top Offenders
```python
-- subquery for # of questions from U
WITH questions_count AS (
  SELECT owner_user_id, COUNT(*) AS val
  FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
  GROUP BY q.owner_user_id)
SELECT answer.owner_user_id as answered_by, question.owner_user_id as question_by,
       COUNT(*) AS pair_count,
       COUNT(*) / questions_count.val as ratio
FROM questions_count,
     `bigquery-public-data.stackoverflow.posts_answers` AS answer
       JOIN `bigquery-public-data.stackoverflow.posts_questions` AS question
       ON question.id = answer.parent_id AND question.owner_user_id = questions_count.owner_user_id
GROUP BY answer.owner_user_id, question.owner_user_id, questions_count.val
ORDER BY ratio DESC
LIMIT 5;
```
![5.6.5](5.6.5.png)


#### Task 5.6.6 Refinements (Written Question)     

1. User will be falsely considered fraudulent in Query 5 if user U and V are in the same high level industrial field, which means their focus would be the same field and not many people have the knowledge to answer questions in this field, as a result the high answer ratio should not be an indicater of reputation farming.    
2. User will be falsely considered innocent in Query 5 if they ask many many questions (let's say 10000 questions), and only 5% of them are answered (small percentage, but 500 answers still). Even though the ratio is low, the person is still doing reputation farming.   
3. We should consider the voted answer into account. For example, we should consider what is the ratio of questions asked by U and answered by V is voted as a preferred answer. This would make sure the answers are high quality, which could potentially eliminate the reputation farming. If the ratio is relatively large, then we could consider the user as top audience; and if the ritio is samll, then we can consider the user's behavior as reputation farming.